In [4]:
import pandas as pd

# User features
users = pd.DataFrame({
    'user_id': [1, 2, 3],
    'age': [25, 30, 22],
    'gender': ['M', 'F', 'M']
})

# Video features
videos = pd.DataFrame({
    'video_id': [101, 102, 103],
    'category': ['comedy', 'drama', 'action'],
    'duration': [10, 20, 15]
})

# Interaction logs
interactions = pd.DataFrame({
    'user_id': [1, 2, 3],
    'video_id': [101, 102, 103],
    'watch_time': [5, 10, 15],
    'avg_dwell_time': [3, 7, 10]
})


In [5]:
interactions

,user_id,video_id,watch_time,avg_dwell_time
0,1,101,5,3
1,2,102,10,7
2,3,103,15,10


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model

# Shared bottom layer
def shared_bottom(input_shape):
    inputs = layers.Input(shape=input_shape)
    x = layers.Dense(64, activation='relu')(inputs)
    return inputs, x

# Task-specific layers
def task_tower(shared_layer, task_name):
    x = layers.Dense(32, activation='relu')(shared_layer)
    outputs = layers.Dense(1, activation='sigmoid', name=task_name)(x)
    return outputs

# Constructing the PLE Model
user_inputs = layers.Input(shape=(3,), name='user_features')  # Example size
video_inputs = layers.Input(shape=(3,), name='video_features')
concat_features = layers.Concatenate()([user_inputs, video_inputs])

shared_inputs, shared_layer = shared_bottom(concat_features.shape[1:])
task1_output = task_tower(shared_layer, 'task_5s_playback')
task2_output = task_tower(shared_layer, 'task_avg_dwell_time')

model = Model(inputs=[user_inputs, video_inputs], outputs=[task1_output, task2_output])

# Compile and summarize the model
model.compile(optimizer='adam',
              loss={'task_5s_playback': 'binary_crossentropy', 
                    'task_avg_dwell_time': 'mse'},
              metrics={'task_5s_playback': 'accuracy', 
                       'task_avg_dwell_time': 'mae'})
model.summary()


In [ ]:
import numpy as np

# Simulated training data
user_data = np.random.rand(100, 3)
video_data = np.random.rand(100, 3)
task_5s_labels = np.random.randint(2, size=(100, 1))
task_avg_dwell_time_labels = np.random.rand(100, 1)

model.fit(
    [user_data, video_data], 
    {'task_5s_playback': task_5s_labels, 'task_avg_dwell_time': task_avg_dwell_time_labels},
    epochs=10, batch_size=32
)